Connection datawarehouse

In [2]:
import pandas as pd 
import pyodbc
import warnings
warnings.simplefilter('ignore')
import sqlite3


DB = {
    'servername': 'tcp:dwhunitedoutdoors.database.windows.net,1433',
    'database': 'DWH_United_Outdoors',
    'username': 'onderzoeker',
    'password': 'Wachtwoord1.' 
}

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=' + DB['servername'] + ';'
    'DATABASE=' + DB['database'] + ';'
    'UID=' + DB['username'] + ';'
    'PWD=' + DB['password'] + ';'
    'Encrypt=yes;'
    'TrustServerCertificate=no;'
    'Connection Timeout=30;'
)

export_conn = pyodbc.connect(conn_str)
export_cursor = export_conn.cursor()
export_cursor



Connection with aenc

In [6]:
database_path = r'C:\Users\\troy\\Desktop\\DAD\\Project folder\\deds_project\\deds\\aenc.accdb'

# Create the connection string
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=' + database_path + ';'
)

# Connect to the database
AENNC_conn = pyodbc.connect(conn_str)

In [27]:


# Write the SQL query
queryBonus = 'SELECT * FROM Bonus'
queryDepartment= 'SELECT * FROM department'
queryRegion= 'SELECT * FROM region'
querySales_order= 'SELECT * FROM sales_order'
querySales_Order_item= 'SELECT * FROM sales_order_item'
queryState= 'SELECT * FROM state'


# Execute the query and fetch the data into a DataFrame
Bonus_df = pd.read_sql_query(queryBonus, AENNC_conn)
Department_df = pd.read_sql_query(queryDepartment, AENNC_conn)

Region_df = pd.read_sql_query(queryRegion, AENNC_conn)

Sales_order_df = pd.read_sql_query(querySales_order, AENNC_conn)

Sales_order_item_df = pd.read_sql_query(querySales_Order_item, AENNC_conn)
State_df = pd.read_sql_query(queryState, AENNC_conn)

# Bonus_df
# Department_df
# Region_df
Sales_order_df
# Sales_order_item_df
# State_df




,id,cust_id,order_date,region,sales_rep
0,2001,101,14-Mar-1996 12:00:00 AM,Eastern,299
1,2002,102,18-Mar-1996 12:00:00 AM,Eastern,467
2,2003,103,21-Mar-1996 12:00:00 AM,Eastern,1039
3,2004,104,23-Mar-1996 12:00:00 AM,South,902
4,2005,101,24-Mar-1996 12:00:00 AM,Eastern,856
...,...,...,...,...,...
645,2650,175,26-Nov-1998 12:00:00 AM,Canada,1596
646,2651,133,26-Nov-1998 12:00:00 AM,Western,299
647,2652,174,26-Nov-1998 12:00:00 AM,Central,902
648,2653,102,17-Jan-1999 12:00:00 AM,Eastern,148


Insert DF in datawarehouse

In [16]:
def insert_into_bonus(Bonus_df, export_conn):
    try:
        export_cursor = export_conn.cursor()

        create_table_query = """
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Bonus' AND xtype='U')
        CREATE TABLE Bonus (
            emp_id NVARCHAR(255),
            bonus_date DATE,
            bonus_amount FLOAT
        )
        """
        export_cursor.execute(create_table_query)

        for index, row in Bonus_df.iterrows():
            try:
                query = """
                INSERT INTO Bonus (emp_id, bonus_date, bonus_amount)
                VALUES (?, ?, ?)
                """
                params = (
                    str(row['emp_id']),
                    row['bonus_date'],
                    float(row['bonus_amount']) if pd.notna(row['bonus_amount']) else None
                )
                export_cursor.execute(query, params)
            except Exception as e:
                print("Error inserting row:", e)
                print("Row data:", row)

        export_conn.commit()
        print("Data inserted successfully into Bonus table.")
    except Exception as e:
        print("Error:", e)
    finally:
        export_cursor.close()

insert_into_bonus(Bonus_df, export_conn)


Data inserted successfully into Bonus table.


In [17]:
def insert_into_department(Department_df, export_conn):
    try:
        export_cursor = export_conn.cursor()

        create_table_query = """
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Department' AND xtype='U')
        CREATE TABLE Department (
            dept_id NVARCHAR(255),
            dept_name NVARCHAR(255),
            dept_head_id NVARCHAR(255)
        )
        """
        export_cursor.execute(create_table_query)

        for index, row in Department_df.iterrows():
            try:
                query = """
                INSERT INTO Department (dept_id, dept_name, dept_head_id)
                VALUES (?, ?, ?)
                """
                params = (
                    str(row['dept_id']),
                    str(row['dept_name']) if pd.notna(row['dept_name']) else None,
                    str(row['dept_head_id']) if pd.notna(row['dept_head_id']) else None
                )
                export_cursor.execute(query, params)
            except Exception as e:
                print("Error inserting row:", e)
                print("Row data:", row)

        export_conn.commit()
        print("Data inserted successfully into Department table.")
    except Exception as e:
        print("Error:", e)
    finally:
        export_cursor.close()

insert_into_department(Department_df, export_conn)


Data inserted successfully into Department table.


In [18]:
def insert_into_region(Region_df, export_conn):
    try:
        export_cursor = export_conn.cursor()

        create_table_query = """
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Region' AND xtype='U')
        CREATE TABLE Region (
            region NVARCHAR(255)
        )
        """
        export_cursor.execute(create_table_query)

        for index, row in Region_df.iterrows():
            try:
                query = """
                INSERT INTO Region (region)
                VALUES (?)
                """
                params = (
                    str(row['region']),
                )
                export_cursor.execute(query, params)
            except Exception as e:
                print("Error inserting row:", e)
                print("Row data:", row)

        export_conn.commit()
        print("Data inserted successfully into Region table.")
    except Exception as e:
        print("Error:", e)
    finally:
        export_cursor.close()

insert_into_region(Region_df, export_conn)


Error inserting row: ('42S22', "[42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name 'region'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Statement(s) could not be prepared. (8180)")
Row data: region    Canada
Name: 0, dtype: object
Error inserting row: ('42S22', "[42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name 'region'. (207) (SQLExecDirectW)")
Row data: region    Central
Name: 1, dtype: object
Error inserting row: ('42S22', "[42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name 'region'. (207) (SQLExecDirectW)")
Row data: region    Eastern
Name: 2, dtype: object
Error inserting row: ('42S22', "[42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name 'region'. (207) (SQLExecDirectW)")
Row data: region    South
Name: 3, dtype: object
Error inserting row: ('42S22', "[42S22] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Inv

In [28]:
def insert_into_sales_order(Sales_order_df, export_conn):
    try:
        export_cursor = export_conn.cursor()

        create_table_query = """
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Sales_Order' AND xtype='U')
        CREATE TABLE Sales_Order (
            id NVARCHAR(255),
            cust_id NVARCHAR(255),
            order_date DATE,
            region NVARCHAR(255),
            sales_rep NVARCHAR(255)
        )
        """
        export_cursor.execute(create_table_query)

        for index, row in Sales_order_df.iterrows():
            try:
                query = """
                INSERT INTO Sales_Order (id, cust_id, order_date, region, sales_rep)
                VALUES (?, ?, ?, ?, ?)
                """
                params = (
                    str(row['id']),
                    str(row['cust_id']),
                    row['order_date'],
                    str(row['region']) if pd.notna(row['region']) else None,
                    str(row['sales_rep']) if pd.notna(row['sales_rep']) else None
                )
                export_cursor.execute(query, params)
            except Exception as e:
                print("Error inserting row:", e)
                print("Row data:", row)

        export_conn.commit()
        print("Data inserted successfully into Sales_Order table.")
    except Exception as e:
        print("Error:", e)
    finally:
        export_cursor.close()


insert_into_sales_order(Sales_order_df, export_conn)


Data inserted successfully into Sales_Order table.


In [21]:
def insert_into_sales_order_item(Sales_order_item_df, export_conn):
    try:
        export_cursor = export_conn.cursor()

        create_table_query = """
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='Sales_Order_Item' AND xtype='U')
        CREATE TABLE Sales_Order_Item (
            id NVARCHAR(255),
            line_id NVARCHAR(255),
            prod_id NVARCHAR(255),
            quantity INT,
            ship_date DATE
        )
        """
        export_cursor.execute(create_table_query)

        for index, row in Sales_order_item_df.iterrows():
            try:
                query = """
                INSERT INTO Sales_Order_Item (id, line_id, prod_id, quantity, ship_date)
                VALUES (?, ?, ?, ?, ?)
                """
                params = (
                    str(row['id']),
                    str(row['line_id']),
                    str(row['prod_id']),
                    int(row['quantity']) if pd.notna(row['quantity']) else None,
                    row['ship_date']
                )
                export_cursor.execute(query, params)
            except Exception as e:
                print("Error inserting row:", e)
                print("Row data:", row)

        export_conn.commit()
        print("Data inserted successfully into Sales_Order_Item table.")
    except Exception as e:
        print("Error:", e)
    finally:
        export_cursor.close()

insert_into_sales_order_item(Sales_order_item_df, export_conn)


Data inserted successfully into Sales_Order_Item table.


In [ ]:
def insert_into_state(State_df, export_conn):
    try:
        export_cursor = export_conn.cursor()

        create_table_query = """
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='State' AND xtype='U')
        CREATE TABLE State (
            state_id NVARCHAR(255),
            state_name NVARCHAR(255),
            state_capital NVARCHAR(255),
            country NVARCHAR(255),
            region NVARCHAR(255)
        )
        """
        export_cursor.execute(create_table_query)

        for index, row in State_df.iterrows():
            try:
                query = """
                INSERT INTO State (state_id, state_name, state_capital, country, region)
                VALUES (?, ?, ?, ?, ?)
                """
                params = (
                    str(row['state_id']),
                    str(row['state_name']) if pd.notna(row['state_name']) else None,
                    str(row['state_capital']) if pd.notna(row['state_capital']) else None,
                    str(row['country']) if pd.notna(row['country']) else None,
                    str(row['region']) if pd.notna(row['region']) else None
                )
                export_cursor.execute(query, params)
            except Exception as e:
                print("Error inserting row:", e)
                print("Row data:", row)

        export_conn.commit()
        print("Data inserted successfully into State table.")
    except Exception as e:
        print("Error:", e)
    finally:
        export_cursor.close()

insert_into_state(State_df, export_conn)
